In [5]:
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
pc = Pinecone(os.getenv("PINECONE_API_KEY"))

pc.create_index(
    name='rag',
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '160452d75f905820c1aa227364b36b3a', 'Date': 'Wed, 28 Aug 2024 23:59:29 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [8]:
import json
data = json.load(open('reviews.json'))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Introduction to Psychology',
  'stars': 4,
  'review': 'Dr. Smith is knowledgeable and engaging. He makes complex topics easy to understand.'},
 {'professor': 'Dr. Emily Johnson',
  'subject': 'Advanced Mathematics',
  'stars': 5,
  'review': 'Dr. Johnson is an excellent professor. She provides clear explanations and encourages questions.'},
 {'professor': 'Prof. Michael Brown',
  'subject': 'History of Art',
  'stars': 3,
  'review': 'Prof. Brown is passionate about the subject, but his lectures can be a bit disorganized.'},
 {'professor': 'Dr. Sarah Davis',
  'subject': 'Computer Science 101',
  'stars': 4,
  'review': 'Dr. Davis is very approachable and always willing to help students outside of class.'},
 {'professor': 'Prof. David Wilson',
  'subject': 'Organic Chemistry',
  'stars': 2,
  'review': 'Prof. Wilson is knowledgeable, but his teaching style is difficult to follow.'},
 {'professor': 'Dr. Jessica Martinez',
  'subject': 'Mic

In [9]:
processed_data = []
client = OpenAI()

for record in data["reviews"]:
    response = client.embeddings.create(
        input=record['review'],
        model='text-embedding-3-small'
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": record["professor"],
        "metadata": {
            "review": record["review"],
            "subject": record['subject'],
            "stars": record["stars"]

        }
    })

In [12]:
processed_data[0]

{'values': [-0.011534428,
  0.0071501518,
  -0.03926614,
  0.016900359,
  0.023267841,
  0.0007084653,
  0.011647186,
  0.034410935,
  0.028786326,
  -0.024408681,
  0.03024554,
  -0.015295222,
  -0.017364655,
  0.00880835,
  0.04558056,
  -0.004112332,
  -0.046137713,
  -0.012184442,
  0.0270618,
  0.034278277,
  0.04396216,
  -0.014446225,
  0.03637424,
  -0.020853505,
  -0.04064576,
  -0.031094538,
  0.024342353,
  0.04083148,
  0.014459491,
  0.0046728035,
  0.06118089,
  -0.014432959,
  -0.016542187,
  0.0016358128,
  -0.012025255,
  0.018266713,
  -0.03544565,
  -3.0378715e-06,
  -0.005203427,
  0.00918642,
  0.056299154,
  0.016223814,
  -0.039717168,
  -0.009418568,
  0.0573604,
  -0.039080422,
  -0.00272442,
  -0.029714916,
  0.032898657,
  0.033959903,
  -0.043139692,
  0.026743425,
  0.017484045,
  -0.037408955,
  -0.049745955,
  0.033508874,
  0.020243287,
  0.035764024,
  -0.0069047385,
  -0.017484045,
  0.04812755,
  0.009988988,
  -0.011852803,
  -0.003535279,
  -0.03491

In [10]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 19}

In [11]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 19}},
 'total_vector_count': 19}